# Secondary Experiment: Generatng Ambiguous Text Examples 

In [ ]:
# importing libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import anthropic
import transformers


In [ ]:
import google.generativeai as genai


### Using Gemini AI

In [ ]:
genai.configure(api_key="XXXXX") # api token authentication 

In [ ]:
# Loading the model 
model = genai.GenerativeModel('gemini-2.0-flash')


In [ ]:
# Constructing  prompt
prompt = """
Generate 30 ambiguous sentences where a pronoun could refer to either an entity or an event.
Here are some examples:
'The volcano erupted violently. It created a huge crater.'
'The bird sang perfectly. It made everyone in the room very happy.'
'Garden was blooming with flowers, which made me feel refreshed.'
Now, generate more examples:
"""
# using model 
response = model.generate_content(prompt)

# getting the generate dexamples 
print(response.text)


Okay, here are 30 ambiguous sentences where a pronoun could refer to either an entity (a person, place, or thing) or an event (an action, situation, or occurrence):

1.  The dog barked loudly at the mailman. It startled the children.
2.  The company laid off a hundred employees. It caused a lot of anger.
3.  She practiced the piano for hours. It exhausted her.
4.  The rain poured down all afternoon. It flooded the streets.
5.  The chef burned the sauce. It ruined the entire meal.
6.  He fixed the leaky faucet carefully. It saved a lot of water.
7.  The politician gave a fiery speech. It swayed many voters.
8.  The child broke the window with a baseball. It was a mistake.
9.  The team won the championship. It was an incredible achievement.
10. The artist painted a beautiful portrait. It impressed all who saw it.
11. The car crashed into the tree. It was a terrifying moment.
12. The student cheated on the test. It resulted in suspension.
13. The baker made a delicious cake. It tempted ev

In [ ]:
# storing results in file 
folder_path = 'output/'  # Replace 'MyFolder' with your folder name

file_path = folder_path + 'examples.txt'


with open(file_path, "w") as file:
    file.write(response.text)
print(f"Output saved to {file_path}")

### Using Claude AI

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key="XXXXX")

response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    # Construct the prompt

    messages=[
        {"role": "user", "content": """\n\nHuman:
    Generate 30 ambiguous sentences where a pronoun could refer to either an entity or an event.
    Here are some examples:
    'The volcano erupted violently. It created a huge crater.'
    'The bird sang perfectly. It made everyone in the room very happy.'
    'Garden was blooming with flowers, which made me feel refreshed.'
    Now, generate more examples:
    \n\nAssistant:"""}
    ]
)

print(response.content[0].text)


Here are 30 ambiguous sentences where a pronoun could refer to either an entity or an event:

1. The team won the championship game. It was an incredible achievement.
2. The politician gave a speech at the rally. It was well-received by the crowd.
3. The family went on a picnic in the park. It was a beautiful day.
4. The student submitted the assignment on time. It earned them a high grade.
5. The hurricane swept through the coastal town. It caused significant damage.
6. The artist painted a stunning landscape. It was showcased in a prestigious gallery.
7. The company launched a new product line. It was met with positive reviews.
8. The couple got married in a beautiful ceremony. It was a joyous occasion.
9. The movie received critical acclaim. It was a box office success.
10. The firefighters battled the blaze for hours. It was a heroic effort.
11. The scientist conducted groundbreaking research. It was published in a prestigious journal.
12. The community hosted a music festival. It 

In [ ]:
# Extract the text content from the response
text_content = response.content[0].text

# Specify the file path
# Create the file path
folder_path = '/content/drive/My Drive/Thesis_gemini/output_folder/'
file_path = folder_path + 'examples.txt'
 # Or your desired path

# Save the text content to the file
with open(file_path, "a") as file:
    file.write(text_content)

print(f"Response saved to {file_path}")

### Using Mistral AI 

In [ ]:

## using Mistral AI from huggingface models 
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# api key 
hf_token = "XXXXX"
#os.environ["HUGGING_FACE_HUB_TOKEN"] = hf_token

# Set device explicitly (optional)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)

# Loading tokenizer and model; configuring model setting 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",  
    use_auth_token=hf_token
)



# Prompting to generate examples 
prompt = """
Generate 30 ambiguous sentences where a pronoun could refer to either an entity or an event. The text example should contain two sentences. First one should be a statement about an entity or an event like "The Volcano erupted violently' and second one should be like 'It created a huge crater". Do not include extra mentions or commentary, I just need ambiguous examples only.
Here are some examples:
'The volcano erupted violently. It created a huge crater.'
'The bird sang perfectly. It made everyone in the room very happy.'
'Garden was blooming with flowers, which made me feel refreshed.'
Now, generate more examples:
"""

# tokenizing the prompt (input) 
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# model fitting 
outputs = model.generate(
    **inputs,
    max_new_tokens=700,
    temperature=0.7,
    do_sample=True
)

# getting generated examples 
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
# storing results 
folder_path = 'output'  
content = generated_text

file_path = folder_path + 'examples_1.txt'


with open(file_path, "w") as file:
    file.write(content)
print(f"Output saved to {file_path}")

### Using Llama 

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"  
hf_token = "XXXXX"

# Load model and tokenizer: configuring model setting 
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto", use_auth_token=hf_token)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# using model pipeline to prompt model to generate text examples 
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token=hf_token
)
messages=[
        {"role": "user", "content": """\n\nHuman:
    Generate 30 ambiguous sentences where a pronoun could refer to either an entity or an event.
    Here are some examples:
    'The volcano erupted violently. It created a huge crater.'
    'The bird sang perfectly. It made everyone in the room very happy.'
    'Garden was blooming with flowers, which made me feel refreshed.'
    Now, generate more examples:
    \n\nAssistant:"""}
    ]
outputs = pipeline(messages, max_new_tokens=700, temperature=0.7, do_sample=True, top_p=0.9)
# displaying output 
print(outputs[0]["generated_text"][-1]['content'])

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 71.33it/s]
Some parameters are on the meta device because they were offloaded to the disk and cpu.
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 47.36it/s]
Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here are 30 ambiguous sentences where a pronoun could refer to either an entity or an event:

1. The storm raged all night. It left a trail of destruction.
2. The concert was a huge success. It sold out in hours.
3. The city was hosting a festival. It attracted thousands of visitors.
4. The painting was created by a famous artist. It is now worth millions.
5. The earthquake struck at dawn. It caused widespread panic.
6. The new policy has been implemented. It is expected to improve efficiency.
7. The song was written by a struggling musician. It became a hit single.
8. The company is facing financial difficulties. It may go bankrupt.
9. The fire was extinguished quickly. It did not cause any serious damage.
10. The athlete has been training for months. It paid off with a gold medal.
11. The novel was a bestseller. It was translated into several languages.
12. The company launched a new product. It has been a huge success.
13. The student was struggling with the coursework. It was too c

In [ ]:
# storing results 
folder_path = 'output'  
results_ = outputs[0]["generated_text"][-1]['content']

file_path = folder_path + 'examples_1.txt'

with open(file_path, "a") as file:
    file.write(results_)
print(f"Output saved to {file_path}")

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key="XXXXX")

answer = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    # Construct the prompt

    messages=[
        {"role": "user", "content": """\n\nHuman:
    Generate 2 ambiguous sentences where a pronoun could refer to either an entity or an event.
    Here are some examples:
    'The volcano erupted violently. It created a huge crater.'
    'The bird sang perfectly. It made everyone in the room very happy.'
    'Garden was blooming with flowers, which made me feel refreshed.'
    Now, generate more examples:
    \n\nAssistant:"""}
    ]
)

print(answer.content[0].text)


Here are two ambiguous sentences where a pronoun could refer to either an entity or an event:

1. "The experiment was a complete failure. It was devastating for the research team."
In this sentence, the pronoun "it" could refer to either the experiment itself or the failure of the experiment.

2. "The storm hit the city hard. It caused widespread damage and power outages."
In this sentence, the pronoun "it" could refer to either the storm itself or the impact of the storm on the city.


In [1]:
# Note: AI Assistants were used to debug and refine the code. 